# Convolutional autoencoder for image denoising

**Author:** [Santiago L. Valdarrama](https://twitter.com/svpino)<br>
**Date created:** 2021/03/01<br>
**Last modified:** 2021/03/01<br>
**Description:** How to train a deep convolutional autoencoder for image denoising.

## Introduction

This example demonstrates how to implement a deep convolutional autoencoder
for image denoising, mapping noisy digits images from the MNIST dataset to
clean digits images. This implementation is based on an original blog post
titled [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html)
by [François Chollet](https://twitter.com/fchollet).

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model


def preprocess(array):
    """
    Normalizes the supplied array and reshapes it into the appropriate format.
    """

    array = array.astype("float32") / 255.0
    array = np.reshape(array, (len(array), 28, 28, 1))
    return array


def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """

    noise_factor = 0.4
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)


def display(array1, array2):
    """
    Displays ten random images from each one of the supplied arrays.
    """

    n = 10

    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]

    plt.figure(figsize=(20, 4))
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    plt.show()


## Prepare the data

In [ ]:
# Since we only need images from the dataset to encode and decode, we
# won't use the labels.
(train_data, _), (test_data, _) = mnist.load_data()

# Normalize and reshape the data
train_data = preprocess(train_data)
test_data = preprocess(test_data)

# Create a copy of the data with added noise
noisy_train_data = noise(train_data)
noisy_test_data = noise(test_data)

# Display the train data and a version of it with added noise
display(train_data, noisy_train_data)

## Build the autoencoder

We are going to use the Functional API to build our convolutional autoencoder.

In [ ]:
input = layers.Input(shape=(28, 28, 1))

# Simplest architecture
# x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
# x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
# x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
# x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
# x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Encoder
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2, 2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

In [ ]:
autoencoder.fit(
    x=noisy_train_data,
    y=train_data,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(noisy_test_data, test_data),
)

Let's now predict on the noisy data and display the results of our autoencoder.

Notice how the autoencoder does an amazing job at removing the noise from the
input images.

In [ ]:
predictions = autoencoder.predict(noisy_test_data)
display(noisy_test_data, predictions)

Save some pictures

In [ ]:
from skimage import io
from skimage.transform import resize
print(train_data[1000].shape)

io.imsave("test.png", noise(train_data[3000][:, :, 0]))
io.imsave("test2.png", noise(resize(train_data[1000][:, :, 0], (256,256))))

Save model

In [ ]:
ouf = open("model.txt", "w")
for i, layer in enumerate(autoencoder.layers[1:]):
    weights = layer.get_weights()
    if weights:
        for x in weights[0].flatten():
            ouf.write( "{:.10f}\n".format(x) )
        for x in weights[1].flatten():
            ouf.write( "{:.10f}\n".format(x) )
        print(weights[0].shape)
        print(weights[1].shape)
ouf.close()

Look at transposed conv operation

In [ ]:
input = layers.Input(shape=(2, 2, 1))

# Encoder
x = layers.Conv2DTranspose(1, (3, 3), strides=2, activation="relu", padding="same")(input)

# Autoencoder
m = Model(input, x)
m.summary()

In [ ]:
w = np.array(
    [
      [10, 2, 3],
     [2, 3, 4],
     [3, 4, 5]
    ]
).reshape((3, 3, 1, 1))
b = np.array([0])
m.set_weights([w, b])

In [ ]:
m.layers[1].get_weights()[0]

In [ ]:
inp = np.array([
                [1, 2],
                [3, 4],
]).reshape(1, 2, 2, 1)
res = m.predict(inp)

In [ ]:
res.reshape((4, 4))

Look at conv operation

In [ ]:
input = layers.Input(shape=(3, 3, 1))

# Encoder
x = layers.Conv2D(1, (3, 3), activation="relu", padding="same")(input)

# Autoencoder
m = Model(input, x)
m.summary()

In [ ]:
w = np.array(
    [
      [1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]
    ]
).reshape((3, 3, 1, 1))
b = np.array([0])
m.set_weights([w, b])

In [ ]:
m.layers[1].get_weights()[0]

In [ ]:
inp = np.array([
                [1, 0, 0],
                [0, 0, 1],
                [0, 1, 0]
]).reshape(1, 3, 3, 1)
res = m.predict(inp)

In [ ]:
res.reshape((3, 3))